<a href="https://colab.research.google.com/github/LienNguyen2912/Japanese-BERT-Model/blob/main/TohokuBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install necessary libraries

In [1]:
pip install transformers

In [2]:
pip install torch

In [8]:
!pip install fugashi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 5.8 MB/s eta 0:00:00


In [10]:
!pip install mecab-python3
!pip install unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=451a241dddf162d196f043082dea365f42568f38b23b58c997aea03eff891b09
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic-lite


In [6]:
!nvidia-smi

Thu Nov 23 03:14:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    30W /  70W |   2241MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
import torch
import time
#from transformers import BertTokenizer, BertModel
from transformers import BertJapaneseTokenizer, BertModel
# Load the Tohoku's Japanese BERT model
model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Set the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print (device)

# Define a function to classify a text and get CLS's embedding
def get_cls_embedding_and_tokenized_output(input_corpus):
    # Tokenize input and get token outputs
    encoded_input_corpus  = tokenizer(input_corpus, return_tensors="pt", truncation=True)
    encoded_input_corpus = {key: val.to(device) for key, val in encoded_input_corpus.items()}

    # Run the model and get CLS's embedding
    start_time = time.time()
    with torch.no_grad():
        output  = model(**encoded_input_corpus )
        cls_embedding = output.last_hidden_state[:, 0]
        # Convert the tokenized output to a list of tokens
        tokenized_output = tokenizer.convert_ids_to_tokens(encoded_input_corpus["input_ids"].squeeze().cpu().numpy())
    elapsed_time = time.time() - start_time
    return cls_embedding, tokenized_output, elapsed_time, encoded_input_corpus

#input_corpus = "これは日本語のテキストのコープスです。"
input_corpus = """
[オペレーター] : こんにちは！お電話ありがとうございます。私の名前はエミリーで、今日はお手伝いできて嬉しいです。どのようにお手伝いできますか？
[ユーザー] : こんにちは、エミリー。貴社のサービスに興味がありますが、それが私にとって適しているかどうか分かりません。
[オペレーター] : 完全に理解していますし、興味を持っていただきありがとうございます！様々なニーズに対応した幅広いサービスを提供しています。サービスに関してもう少し教えていただけますか？
[ユーザー] : そうですね、信頼性があり、手頃な価格のものが必要です。過去にサービスで悪い経験をしました。
[オペレーター] : 了解しました。当社はすべてのサービスで信頼性と手頃な価格を重視しています。パッケージについての詳細を共有させていただきます。現在、新規顧客向けに割引料金が適用される特別なプロモーションを実施しています。これにより、高品質なサービスを手頃な価格でご利用いただけます。
[ユーザー] : 興味深いですね。サービスの具体的な機能について教えていただけますか？
[オペレーター] : もちろんです！当社のサービスには [機能1]、[機能2]、および [機能3] が含まれており、お客様の特定のニーズに応じて設計されています。これらの機能は非常に有益だと感じている現在のお客様からも優れたフィードバックをいただいています。
[ユーザー] : うーん、有望ですね。でもまだ自分に合っているかどうか分かりません。
[オペレーター] : お気持ち理解しています。迷われているのは分かります。お客様の決断をサポートするために、30日間の返金保証もご用意しています。これにより、1か月間はリスクを取らずにサービスを試していただけます。期待通りでない場合は、全額返金いたします。お気軽にお試しください。
[ユーザー]: それは寛大な提案ですね。保証について感謝します。
"""
cls_embedding, tokenized_output, elapsed_time, encoded_input_corpus = get_cls_embedding_and_tokenized_output(input_corpus)

print("CLS embedding's first 20 elements:", cls_embedding.squeeze()[:20].tolist())
print("CLS embedding size:", cls_embedding.size())
print("Inference time:", elapsed_time)
print("Input corpus: ", input_corpus)
print("Tokenized output:", tokenized_output)
print("Tokenized output length:", len(tokenized_output))
# Print encoded_input_corpus properties
print("Input IDs:", encoded_input_corpus["input_ids"])
print("Attention Mask:", encoded_input_corpus["attention_mask"])
print("Token Type IDs:", encoded_input_corpus["token_type_ids"])

cuda
CLS embedding's first 20 elements: [0.8804212808609009, -0.09973394125699997, -0.5113552808761597, 0.05032897740602493, -0.07233496010303497, -0.16651344299316406, -0.24157138168811798, -0.4871055781841278, -0.534979522228241, 0.09847936034202576, 0.7304293513298035, -0.2848810851573944, 0.6309994459152222, 0.1546383500099182, -0.32958075404167175, -0.4615398943424225, 0.23815754055976868, -0.22955204546451569, 0.17011003196239471, -0.05814847722649574]
CLS embedding size: torch.Size([1, 768])
Inference time: 0.051265716552734375
Input corpus:  
[オペレーター] : こんにちは！お電話ありがとうございます。私の名前はエミリーで、今日はお手伝いできて嬉しいです。どのようにお手伝いできますか？
[ユーザー] : こんにちは、エミリー。貴社のサービスに興味がありますが、それが私にとって適しているかどうか分かりません。
[オペレーター] : 完全に理解していますし、興味を持っていただきありがとうございます！様々なニーズに対応した幅広いサービスを提供しています。サービスに関してもう少し教えていただけますか？
[ユーザー] : そうですね、信頼性があり、手頃な価格のものが必要です。過去にサービスで悪い経験をしました。
[オペレーター] : 了解しました。当社はすべてのサービスで信頼性と手頃な価格を重視しています。パッケージについての詳細を共有させていただきます。現在、新規顧客向けに割引料金が適用される特別なプロモーションを実施しています。これにより、高品質なサービスを手頃な価格でご利用いただけます。
[ユーザー] : 興味深い